In [1]:
import math
import nltk
from nltk.lm.models import Lidstone
from nltk.lm.models import Laplace
from nltk.corpus import machado
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import re

In [113]:
def remover_stopwords(texto):
    # Carregue a lista de stopwords da língua desejada (exemplo: português)
    stopwords_lista = nltk.corpus.stopwords.words('portuguese')
    
    # Tokenize o texto em palavras
    palavras = word_tokenize(texto)
    
    # Crie uma lista de palavras sem as stopwords
    palavras_sem_stopwords = [palavra for palavra in palavras if palavra.lower() not in stopwords_lista]
    
    # Recrie o texto sem as stopwords
    texto_sem_stopwords = ' '.join(palavras_sem_stopwords)
    
    return texto_sem_stopwords

def generate_ngrams(text, n):
    words = text.split()
    bigrams = []
    for i in range(len(words) - n + 1):
        bigram =  " ".join(words[i:i+n]) 
        bigrams.append(tuple(bigram.split())) 
    
    return bigrams

def remocao_caracteres_especiais(text):
    padrao = r'[^\w]+'
    return re.sub(padrao, ' ', text)

def trans_corpus_text(corpus):
    text = ''
    for i in corpus:
        text += machado.raw(i) +' '
    
    #Pre-processamento
    text = remover_stopwords(text)
    text = remocao_caracteres_especiais(text)
    
    return text

def criar_modelo(text,n, model=None, gamma=0.1):
    tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(text)]
    train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
    
    if model == 'li':
        mo = Lidstone(gamma,order=n )
    elif model == 'la':
        mo = Laplace(order=n)
    else:
        mo = MLE(n)
        
    mo.fit(train_data, padded_sents)
    return mo

In [20]:
corpus = machado.fileids()
c = trans_corpus_text(corpus)

In [22]:
model = criar_modelo(c,1)

In [45]:
vocabulario = model.vocab.counts
total_palavras = len(c.split())

A)

In [38]:
binario = []
for palavra in vocabulario:
    binario.append(palavra)
binario[:10]

['conto',
 'contos',
 'fluminenses',
 '1870',
 'texto',
 'fonte',
 'obra',
 'completa',
 'machado',
 'assis']

B) 

In [79]:
def contagem(vocabulario):
    dic_contagem = {}
    for palavra in vocabulario:
        dic_contagem[palavra] = vocabulario.get(palavra)
    return dic_contagem
contagem(vocabulario)['se']

17030

C)

In [100]:
def frequencia(vocabulario):
    dic_frequencia = {}
    for palavra in vocabulario:
        frequencia = vocabulario.get(palavra)/total_palavras
        dic_frequencia[palavra] = f'{frequencia: .16f}'
    return dic_frequencia

print(frequencia(vocabulario)['se'])

 0.0118481269106235


D)

In [96]:
def frequencia_inversa(vocabulario):
    dic_frequencia_inversa = {}
    for palavra in vocabulario:
        frequencia = vocabulario.get(palavra)/total_palavras
        frequencia_inversa = frequencia * math.log2(1/vocabulario.get(palavra))
        dic_frequencia_inversa[palavra] = f'{frequencia_inversa: .16f}'
    return dic_frequencia_inversa
frequencia_inversa(vocabulario)['se']
    

'-0.1665347934001532'

E) 

In [123]:
bigram = generate_ngrams(c,2)

In [141]:
len(set(bigram))

1067246

In [128]:
len(bigram)

1437357

In [152]:
from collections import Counter
def frequencia_ngram(ngram):
    contagem1 = Counter(set(ngram))
    contagem2 = Counter(ngram)
    
    # Subtrair a contagem de lista2 da contagem de lista1
    bigram_repetidos = list((contagem2 - contagem1).elements())
    
    
    dic_freq = {}
    for b in set(ngram):
        dic_freq[b] = 1
    
    for b in bigram_repetidos:
        dic_freq[b] += 1 
    
    return dic_freq

In [155]:
frequencia_ngram(bigram)[('Rio','Janeiro')]

709

F)

In [156]:
trigram = generate_ngrams(c,3)
print(len(set(trigram)))
print(len(trigram))

1384866
1437356


In [157]:
trigram_fre = frequencia_ngram(trigram)

In [159]:
maior = 0
maior_nome = ''
for i in trigram_fre:
    if maior < trigram_fre[i]:
        maior = trigram_fre[i]
        maior_nome = i
print(maior_nome, maior)        

('Obra', 'Completa', 'Machado') 144
